UserRepository.kt

In [ ]:
    suspend fun onCustomerCheckOrderStatus(customerCheckOrderStatusParam: CustomerCheckOrderStatusParam): Resource<List<CustomerCheckOrderStatus>>

UserRepositoryImpl.kt

In [ ]:
    override suspend fun onCustomerCheckOrderStatus(customerCheckOrderStatusParam: CustomerCheckOrderStatusParam): Resource<CustomerCheckOrderStatus>{
        var authToken: String = ""
        getUserInfo().collectLatest { authToken = it.authToken }
        return try {
            val response = api.onCustomerCheckOrderStatus(
                CustomerCheckOrderStatusRequest(param = customerCheckOrderStatusParam),
                authToken = "Bearer $authToken"
            )
            if (response.response.status) {
                Resource.Success(response.response.orderStatus)
            } else {
                Resource.Error(response.response.message)
            }
        } catch(e: Exception) {
            e.printStackTrace()
            Resource.Error(e.message ?: "An unknown error occurred.")
        }
    }

UserDeliveryScreen

In [ ]:
    val list:CustomerCheckOrderStatusList by userViewModel.CustomerCheckOrderStatusList
    LazyColumn() {
        items(list.data)
        { item ->
            Text(text = item.chef_first_name)
        }
    }


    LaunchedEffect(Unit){
        while(true) {
            userViewModel.onCustomerCheckOrderStatus()
            delay(5000)
        }
    }

In [ ]:
data class CustomerCheckOrderStatusList(
    val data: List<CustomerCheckOrderStatus>? = emptyList()
)

UserViewModel.kt

In [ ]:
    fun onCustomerCheckOrderStatus() {
        viewModelScope.launch (Dispatchers.IO) {
            when (val result =
                userRepository.onCustomerCheckOrderStatus(CustomerCheckOrderStatusParam(userState.value.userInfo.email))
            ) {
                is Resource.Success -> {
                    val CustomerCheckOrderStatusList = result.data
                    _CustomerCheckOrderStatusList.value =
                        CustomerCheckOrderStatusList(
                            CustomerCheckOrderStatusList
                        )
                }

                is Resource.Error -> {
                Log.d("UserViewModel", "checkCustomerOrderStatus Error ${result.message}")

                }
            }
        }
    }

In [ ]:
    override suspend fun onCustomerCheckOrderStatus(customerCheckOrderStatusParam: CustomerCheckOrderStatusParam
    ): Resource<List<CustomerCheckOrderStatus>> {
        var authToken: String = ""
        getUserInfo().collectLatest { authToken = it.authToken }
        return try {
            val response = api.onCustomerCheckOrderStatus(
                CustomerCheckOrderStatusRequest(param = CustomerCheckOrderStatusParam((email))),
                authToken = "Bearer $authToken"
            )
            if (response.response.status) {
                Resource.Success(response.response.orderStatus)
            } else {
                Resource.Error(response.response.message)
            }
        } catch (e: Exception) {
            e.printStackTrace()
            Resource.Error(e.message ?: "An unknown error occurred.")
        }
    }